# Array Interview Question


### Anagram Check

anagram是一種字的轉換，使用相同的字母以任意順序重新組成不同的字，之中有任意空白都可以例如， "apple" -> "ap e lp"


In [37]:
def anagram(s1, s2):
    l_bound = ord('0')
    r_bound = ord('z')
    appeared = [0]*(r_bound - l_bound)
    
    for letter in s1:
        if letter != ' ':
            mapping = ord(letter) - l_bound
            appeared[mapping] += 1

    for letter in s2:
        if letter != ' ':
            mapping = ord(letter) - l_bound
            appeared[mapping] -= 1
            if appeared[mapping] < 0:
                return False
    
    for ele in appeared:
        if ele != 0:
            return False
    
    return True


In [38]:
import unittest


class TestAnagram(unittest.TestCase):
    
    def test(self, solve):
        
        self.assertEqual(solve('go go go','gggooo'), True)
        self.assertEqual(solve('abc','cba'), True)
        self.assertEqual(solve('hi man','hi     man'), True)
        self.assertEqual(solve('aabbcc','aabbc'), False)
        self.assertEqual(solve('123','1 2'), False)
        print('success')
        

t = TestAnagram('test') # need to provide the method name, default is runTest
t.test(anagram)

success


個人這邊這解法可能會不夠完善，因為僅僅是針對魚數字字母的陣列mapping,但是萬一有符號就不知道要怎辦了，所以當然是可以用dict來解掉這煩人的問題拉，只是想說這是屬於array類別的問題，就故意只用array解

### Array Pair Sum

給予一個數字陣列，找出所有特定的數字配對的加起來為特定值k
ex.

```python

pair_sum([1,3,2,2], 4)

(1,3)
(2,2)

今天是要回傳有幾個配對就好，所以是回傳數字2
```

In [39]:
def pair_sum(arr,k):
    res = [False]*len(arr)
    
    for i in range(len(arr)-1):
        for j in range(i+1,len(arr)):
            if arr[i] + arr[j] == k:
                    res[i] = True
                    res[j] = True
                   
    pair_count = [1 for ele in res if ele == True]
    
    return len(pair_count)//2

上面效率會是$ Big O(n^2) $，但是如果可以使用dict或是set的話，就可以把效率壓到 $ BigO(n) $，因為 `n in dict` 這樣的查找只需 $ BigO(1) $，在array找尋你要的值是要花費 $ BigO(n) $,下面我們就來換成用set or dict來實作

In [40]:
def pair_sum_set_version(arr, k):
    to_seek = set()
    output = set()
    
    for num in arr:
        
        target = k - num
        
        if target not in to_seek:
            to_seek.add(num)
        else:
            output.add((min(num, target), max(num, target)))
            
    return len(output)

In [41]:
class TestPairSum(unittest.TestCase):
    
    def test(self, solve):
        
        self.assertEqual(solve([1,9,2,8,3,7,4,6,5,5,13,14,11,13,-1],10),6)
        self.assertEqual(solve([1,2,3,1],3),1)
        self.assertEqual(solve([1,3,2,2],4),2)
        print('success')
        
t = TestPairSum()
t.test(pair_sum_set_version)

success


### finding missing element

這題是會給予你兩個array，第二個array是從第一個array隨機刪除一個元素後，並且進行洗亂的動作，然後今天你的任務就是要去找那個消失的元素

In [42]:
def finder(ary, ary2):
    table = {}
    
    for ele in ary:
        if ele in table:
            table[ele] += 1
        else:
            table[ele] = 1
    
    for ele in ary2:
        if ele in table:
            table[ele] -= 1
        else:
            return ele
    
    for k, v in table.items():
        if v != 0:
            return k

上面這個邏輯，如果是先用ary2去做表紀錄的話邏輯上會更加簡潔，也會少了最後一步

```python

for ele in ary2:
    table[ele] = 1

for ele in ary1:
    if (ele not in table) or (table[ele] == 0):
        return ele
    else:
        table[ele] -= 1

```

這個解法算是最快的，因為如果使用排序的話最少都會要 $ n \log n $，排序就是loop他去找不一樣的元素而已。


另外有個天殺的聰明解法，這我真的沒想到就是使用XOR，讓我們先來看看code
xor ( exclude or ) 具有排他性的or，就是or只要兩者之一有true結果就會是true，但是兩個都是true對於程式會是一種ambiguous，因此exclude這種情況，所以xor就是one or the other but not both


$ A \vee B $ but not  $ A \wedge B $

直接從語意上翻譯成數學就是像下面

$$ A \oplus B = (A \vee B) \wedge \neg ( A \wedge B) $$


總之呢！ 因為xor的特性，若是兩個完全一樣的ary，你將會發現最後結果會是0

```python

def finder_xor(arr1, arr2): 
    result=0 
    
    # Perform an XOR between the numbers in the arrays
    for num in arr1+arr2: 
        result^=num 
        print result
        
    return result 
    
```



In [43]:
class TestFinder(unittest.TestCase):
    
    def test(self, solve):
        
        self.assertEqual(solve([5,5,7,7],[5,7,7]),5)
        self.assertEqual(solve([1,2,3,4,5,6,7],[3,7,2,1,4,6]),5)
        self.assertEqual(solve([9,8,7,6,5,4,3,2,1],[9,8,7,5,4,3,2,1]),6)
        print('success')
        
t = TestFinder()
t.test(finder)

success


### largest continuous sum

題目會給予你一個陣列，你的任務就是要去從裡面發現哪種連續數字的總和會是最大值，不一定是全部數字加起來是最大，因為裡面會有負數，有可能是從某某位置開始的連續Ｘ個數子總和才是最大。


In [44]:
def lar_con_sum(ary):
    
    if len(ary) == 0:
        return 0
    
    max_sum = cur_sum = ary[0]
    
    for num in ary[1:]:
        cur_sum = max(cur_sum+num, num)
        max_sum = max(cur_sum, max_sum)
        
    return max_sum
        
    

這題的思緒是，長度n的連續數字最大和，一定是從長度n-1連續數字最大和來的

所以今天從index=0時來看，因為元素只有一個這時候就是他本身為最大值，當index=1時，我們就要來比較ele[0]+ele[1]和ele[0] <- 當前最大值的比較，比較這兩者然後取最大的，需要注意的是，我們需要暫存目前的sum，因為這是拿來判斷後面遇到負數狀時況，計算另一個最大值的點，此時另一個最大值(cur_sum)仍然會與之前最大值去比較(max_sum)，

In [45]:
class TestLargestConSum(unittest.TestCase):
    
    def test(self, solve):
        
        self.assertEqual(solve([1,2,-1,3,4,-1]),9)
        self.assertEqual(solve([1,2,-1,3,4,10,10,-10,-1]),29)
        self.assertEqual(solve([-1,1]),1)
        self.assertEqual(solve([1,2,-10,5,6]), 11)
        print('success')
        
t = TestLargestConSum()
t.test(lar_con_sum)

success


#### Sentence Reversal

給予一個字串，然後反轉單字順序，例如： 'here it is' -> 'is it here'

In [46]:
def sentenceReversal(str1):
    str1 = str1.strip()
    words = str1.split()  
    
    result = ''
    
    for i in range(len(words)):
        result += ' '+words[len(words)-i-1]
        
    return result.strip()
        

In [47]:
class TestSentenceReversal(unittest.TestCase):
    
    def test(self, solve):
        self.assertEqual(solve('    space before'),'before space')
        self.assertEqual(solve('space after     '),'after space')
        self.assertEqual(solve('   Hello John    how are you   '),'you are how John Hello')
        self.assertEqual(solve('1'),'1')
        print('success')
        
t = TestSentenceReversal()
t.test(sentenceReversal)

success


值得注意的是python string split這個方法，不帶參數的話，預設是做strip的事然後分割，跟你使用 split(' ')得到的結果會不一樣，另外面試時可能要使用比較基本的方式來實作這題，也就是少用python trick的方式。

#### string compression

給予一串字串，轉換成數字加字母的標記法，雖然覺得這個壓縮怪怪的，因為無法保留字母順序

In [48]:
def compression(str1):
    mapping = {}
    letter_order = [False]
    result = ''
    
    for ele in str1:
        if ele != letter_order[-1]:
            letter_order.append(ele)
            
        if ele not in mapping:
            mapping[ele] = 1
        else:
            mapping[ele] += 1
            
    for key in letter_order[1:]:
        result += '{}{}'.format(key, mapping[key])
        
    return result

In [49]:
class TestCompression(unittest.TestCase):
    
    def test(self, solve):
        self.assertEqual(solve(''), '')
        self.assertEqual(solve('AABBCC'), 'A2B2C2')
        self.assertEqual(solve('AAABCCDDDDD'), 'A3B1C2D5')
        print('success')
        
t = TestCompression()
t.test(compression)

success


#### unique characters in string

給予一串字串並判斷他是否全部不同的字母


In [50]:
def uni_char(str1):
    mapping = {}
    
    for letter in str1:
        if letter in mapping:
            return False
        else:
            mapping[letter] = True
    
    return True

def uni_char2(str1):
    return len(set(str1)) == len(str1)

In [51]:
class TestUniChar(unittest.TestCase):
    
    def test(self, solve):
        self.assertEqual(solve(''), True)
        self.assertEqual(solve('goo'), False)
        self.assertEqual(solve('abcdefg'), True)
        print('success')
        
t = TestUniChar()
t.test(uni_char2)

success
